## Import

In [52]:
# Standard libraries
import re
import string
import random
import pickle
import os
import json
from typing import Optional, Dict
import random
import logging
from enum import Enum
from datetime import datetime, timedelta
import json
from random import choice

# Data processing
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt

# Text preprocessing & NLP
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Machine Learning
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

# Deep Learning - TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from keras.saving import register_keras_serializable
from tensorflow.keras.layers import (
    Input, Embedding, SpatialDropout1D, Bidirectional, LSTM,
    Dense, Dropout, LayerNormalization, Lambda, Concatenate,BatchNormalization
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2


## Preprocessing Data

### Load Data

In [5]:
df = pd.read_csv('../dataset/triplet_dataset.csv')

### Sample Data

In [6]:
print(f"Original dataset shape: {df.shape}")
df.head()

Original dataset shape: (45818, 3)


,anchor,positive,negative
0,Pelanggan bawa motorcycle vespa dengan stang t...,"Saya teknisi, cek motorcycle vespa, ada Busi n...",Bisa atur jadwal servis di bengkel Bandung min...
1,Servis truck saya untuk power steering berat s...,Bagaimana status servis truck saya? Sudah sele...,Estimasi biaya ganti busi untuk motorcycle isu...
2,Servis car saya untuk busi ngaco sudah sampai ...,Kapan perkiraan selesai perbaikan Susah starte...,"Kalau Rem bunyi di truck saya diperbaiki, kira..."
3,Ada saran bengkel di Medan yang jago tangani m...,Ada saran bengkel di Medan yang jago tangani P...,BYsa atur jadwal servis di bengkel Semarang mi...
4,Bagaimana cara booking online untuk perbaikan ...,Saya mau booking servis untuk truck hino yang ...,Mesin goyang pas diam itu normal ga untuk truc...


### Data Cleaning

In [7]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = str(text).lower()
    text = re.sub(r"\n", " ", text)                  
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\s+", " ", text)                 
    return text.strip()

In [8]:
for col in ['anchor', 'positive', 'negative']:
    df[col] = df[col].apply(clean_text)

print("\nSample cleaned data:")
print(df.head())


Sample cleaned data:
                                              anchor  \
0  pelanggan bawa motorcycle vespa dengan stang t...   
1  servis truck saya untuk power steering berat s...   
2  servis car saya untuk busi ngaco sudah sampai ...   
3  ada saran bengkel di medan yang jago tangani m...   
4  bagaimana cara booking online untuk perbaikan ...   

                                            positive  \
0  saya teknisi cek motorcycle vespa ada busi nga...   
1  bagaimana status servis truck saya sudah seles...   
2  kapan perkiraan selesai perbaikan susah starte...   
3  ada saran bengkel di medan yang jago tangani p...   
4  saya mau booking servis untuk truck hino yang ...   

                                            negative  
0  bisa atur jadwal servis di bengkel bandung min...  
1  estimasi biaya ganti busi untuk motorcycle isu...  
2  kalau rem bunyi di truck saya diperbaiki kirak...  
3  bysa atur jadwal servis di bengkel semarang mi...  
4  mesin goyang pas diam itu 

### Triplet Validation

In [9]:
def is_valid_triplet(row):
    return all([
        isinstance(row['anchor'], str) and len(row['anchor'].split()) > 3,
        isinstance(row['positive'], str) and len(row['positive'].split()) > 3,
        isinstance(row['negative'], str) and len(row['negative'].split()) > 3,
    ])

In [10]:
df = df[df.apply(is_valid_triplet, axis=1)]
df = shuffle(df).reset_index(drop=True)

print("Cleaned shape:", df.shape)
df.to_csv('../dataset/cleaned_triplet_dataset.csv', index=False)

Cleaned shape: (45818, 3)


## Tokenization

### Tokenizer Definition

In [11]:
VOCAB_SIZE = 10000    
OOV_TOKEN = "<OOV>"
MAX_SEQUENCE_LENGTH = 30 

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(df['anchor'].tolist() + df['positive'].tolist() + df['negative'].tolist())

### Tokenize Data

In [12]:
def tokenize_and_pad(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    return padded

anchor_input   = tokenize_and_pad(df['anchor'].tolist())
positive_input = tokenize_and_pad(df['positive'].tolist())
negative_input = tokenize_and_pad(df['negative'].tolist())

print("Tokenization done")
print("Anchor sample:", anchor_input[0])
print("Shape of anchor input:", anchor_input.shape)

Tokenization done
Anchor sample: [ 69 183 184 131   2   8   4  49  68  62  18   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
Shape of anchor input: (45818, 30)


### Save Tokenizer

In [13]:
with open("../generated/siamese_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

### Save Tokenized Data

In [14]:
np.save("../generated/anchor_input.npy", anchor_input)
np.save("../generated/positive_input.npy", positive_input)
np.save("../generated/negative_input.npy", negative_input)

## Siamse LSTM Model

In [15]:
MAX_SEQUENCE_LENGTH = 30
VOCAB_SIZE          = 10000
EMBEDDING_DIM       = 128
LSTM_UNITS          = 32   
DENSE_UNITS         = 32
DROPOUT_RATE        = 0.3
MARGIN              = 0.5
LEARNING_RATE       = 1e-4
L2_REG              = 1e-5 

### Load Tokenized Data

In [16]:
anchor_input = np.load("../generated/anchor_input.npy")
positive_input = np.load("../generated/positive_input.npy")
negative_input = np.load("../generated/negative_input.npy")

### Load Tokenizer

In [17]:
with open("../generated/siamese_tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

### Build Encoder

In [18]:
def build_shared_encoder(vocab_size, embedding_dim, seq_len, dropout_rate):
    inp = Input(shape=(seq_len,), name="input_text")
    x = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=seq_len,
        mask_zero=True,
        embeddings_regularizer=l2(L2_REG),
        name="embedding_layer"
    )(inp)
    x = SpatialDropout1D(dropout_rate)(x)

    x = Bidirectional(
        LSTM(LSTM_UNITS, 
             dropout=dropout_rate, 
             recurrent_dropout=dropout_rate,
             kernel_regularizer=l2(L2_REG)
    ), name="bilstm_layer")(x)

    x = BatchNormalization()(x)
    x = Dense(DENSE_UNITS, activation="relu", kernel_regularizer=l2(L2_REG))(x)
    x = Dropout(dropout_rate)(x)
    x = LayerNormalization()(x)
    x = Lambda(lambda t: tf.math.l2_normalize(t, axis=1), name="l2_norm")(x)
    return Model(inp, x, name="shared_encoder")

In [19]:

@register_keras_serializable()
def triplet_loss(y_true, y_pred, margin=0.5):
    
    # Bagi menjadi tiga embedding sepanjang dim-1
    anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)

    # Hitung squared Euclidean distance
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    # Triplet loss
    basic_loss = pos_dist - neg_dist + margin
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
    return loss

### Build Siamse LSTM Model

In [20]:
def build_siamese_model():
    encoder = build_shared_encoder(VOCAB_SIZE, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, DROPOUT_RATE)

    a_in = Input((MAX_SEQUENCE_LENGTH,), name="anchor_input")
    p_in = Input((MAX_SEQUENCE_LENGTH,), name="positive_input")
    n_in = Input((MAX_SEQUENCE_LENGTH,), name="negative_input")

    a_emb = encoder(a_in)
    p_emb = encoder(p_in)
    n_emb = encoder(n_in)

    # Concatenate embeddings on feature axis (not batch axis!)
    merged = Concatenate(axis=1, name="merged_embeddings")([a_emb, p_emb, n_emb])
    return Model(inputs=[a_in, p_in, n_in], outputs=merged, name="siamese_model")

In [21]:
model = build_siamese_model()
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss=triplet_loss
)
model.summary()

c:\Users\Lenovo\.conda\envs\tensorflow_cpu\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "siamese_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ anchor_input        │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positive_input      │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ negative_input      │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_encoder      │ (None, 32)        │  1,323,616 │ anchor_input[0][… │
│ (Functional)        │                   │            │ positive_input[0… │
│                     │                   │            │ negative_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ merged_embeddings   │ (None, 96)        │          0 │ shared_encoder[0… │
│ (Concatenate)       │                   │            │ shared_encoder[1… │
│                     │                   │            │ shared_encoder[2… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,323,616 (5.05 MB)

 Trainable params: 1,323,488 (5.05 MB)

 Non-trainable params: 128 (512.00 B)

In [22]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, min_delta=1e-4, restore_best_weights=True),
    ModelCheckpoint("../generated/siamese_best.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
]

### Training Model

In [23]:
idx = np.arange(anchor_input.shape[0])
train_idx, val_idx = train_test_split(idx, test_size=0.1, random_state=42, shuffle=True)

X_train = {
    'anchor_input':   anchor_input[train_idx],
    'positive_input': positive_input[train_idx],
    'negative_input': negative_input[train_idx],
}
y_train = np.zeros((train_idx.shape[0], 1))

X_val = {
    'anchor_input':   anchor_input[val_idx],
    'positive_input': positive_input[val_idx],
    'negative_input': negative_input[val_idx],
}
y_val = np.zeros((val_idx.shape[0], 1))

In [24]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=128,
    epochs=30,
    shuffle=True,
    callbacks=callbacks
)

Epoch 1/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 80s 163ms/step - loss: 0.3660 - val_loss: 0.0168 - learning_rate: 1.0000e-04
Epoch 2/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 46s 142ms/step - loss: 0.1194 - val_loss: 0.0080 - learning_rate: 1.0000e-04
Epoch 3/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 48s 147ms/step - loss: 0.0731 - val_loss: 0.0059 - learning_rate: 1.0000e-04
Epoch 4/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 47s 145ms/step - loss: 0.0531 - val_loss: 0.0056 - learning_rate: 1.0000e-04
Epoch 5/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 47s 145ms/step - loss: 0.0410 - val_loss: 0.0054 - learning_rate: 1.0000e-04
Epoch 6/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 151ms/step - loss: 0.0359 - val_loss: 0.0052 - learning_rate: 1.0000e-04
Epoch 7/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 50s 155ms/step - loss: 0.0301 - val_loss: 0.0051 - learning_rate: 1.0000e-04
Epoch 8/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 151ms/step - loss: 0.0277 - val_loss: 0.0049 - learning_rate: 1.0000e-04
Epoch 9/30
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 153ms/step - loss: 0

### Save Trained Siamse LSTM Model

In [25]:
# Save full model (with loss config)
model.save("../generated/siamese_lstm_triplet_model.keras")
# Save just encoder for inference
encoder = model.get_layer("shared_encoder")
encoder.save("../generated/siamese_encoder_only.keras")

### Inference Siamse LSTM Model

In [26]:
anchor_embed = encoder.predict(anchor_input)
positive_embed = encoder.predict(positive_input)
negative_embed = encoder.predict(negative_input)

1432/1432 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step


In [27]:
sim_pos = np.array([cosine_similarity([anchor_embed[i]], [positive_embed[i]])[0][0] for i in range(len(anchor_embed))])
sim_neg = np.array([cosine_similarity([anchor_embed[i]], [negative_embed[i]])[0][0] for i in range(len(anchor_embed))])

In [28]:
y_true = np.ones(len(anchor_embed))
y_pred = sim_pos > sim_neg  # True if similarity to positive > negative
accuracy = accuracy_score(y_true, y_pred)

In [29]:
def siamese_inference(anchor_text, positive_text, negative_text):
    # Tokenisasi dan padding
    anchor_seq = tokenizer.texts_to_sequences([anchor_text])
    positive_seq = tokenizer.texts_to_sequences([positive_text])
    negative_seq = tokenizer.texts_to_sequences([negative_text])

    anchor_pad = pad_sequences(anchor_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    positive_pad = pad_sequences(positive_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    negative_pad = pad_sequences(negative_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

    # Dapatkan embedding dari encoder
    anchor_emb = encoder.predict(anchor_pad)
    positive_emb = encoder.predict(positive_pad)
    negative_emb = encoder.predict(negative_pad)

    # Hitung similarity
    sim_pos = cosine_similarity(anchor_emb, positive_emb)[0][0]
    sim_neg = cosine_similarity(anchor_emb, negative_emb)[0][0]

    return {
        "sim_pos": sim_pos,
        "sim_neg": sim_neg,
        "is_positive_closer": sim_pos > sim_neg
    }

result = siamese_inference(
    "saya ingin booking servis mobil besok",
    "bisa booking servis rutin mobil besok?",
    "berapa biaya ganti oli?"
)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
{'sim_pos': 0.64097047, 'sim_neg': -0.21067399, 'is_positive_closer': True}


In [30]:
print(f"Top-1 Matching Accuracy: {accuracy * 100:.2f}%")

Top-1 Matching Accuracy: 100.00%


In [31]:
y_labels = np.concatenate([
    np.ones(len(anchor_embed)),   # positive pair
    np.zeros(len(anchor_embed))   # negative pair
])

In [32]:
similarities = np.concatenate([
    np.array([cosine_similarity([anchor_embed[i]], [positive_embed[i]])[0][0] for i in range(len(anchor_embed))]),
    np.array([cosine_similarity([anchor_embed[i]], [negative_embed[i]])[0][0] for i in range(len(anchor_embed))])
])

In [33]:
threshold = 0.5
y_pred_bin = (similarities > threshold).astype(int)

precision, recall, f1, _ = precision_recall_fscore_support(y_labels, y_pred_bin, average='binary')
auc = roc_auc_score(y_labels, similarities)

In [34]:
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, AUC: {auc:.2f}")

Precision: 1.00, Recall: 1.00, F1 Score: 1.00, AUC: 1.00


## Build Intent Classifier Model (TF-IDF + SVM)

### Load Dataset

In [35]:
df = pd.read_csv("../dataset/merged_intent_dataset.csv")

In [36]:
df.head()

,text,intent
0,"halo bro, apa kabar?",greeting
1,"selamat pagi, siap bantu!",greeting
2,"hai, kendaraanmu baik-baik saja?",greeting
3,"apa kabar, mekanik favoritmu di sini!",greeting
4,"selamat malam, ada yang rusak?",greeting


### Data Preparation

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["intent"], test_size=0.2, stratify=df["intent"], random_state=42
)

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)),
    ("svm", SVC(kernel="linear", probability=True, random_state=42))
])

# 5. Train model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)),
                ('svm',
                 SVC(kernel='linear', probability=True, random_state=42))])

### Train Model

In [38]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

                                     precision    recall  f1-score   support

                            apology       1.00      1.00      1.00        10
                ask_booking_details       1.00      1.00      1.00        10
                 ask_booking_status       1.00      1.00      1.00        10
               ask_bot_capabilities       1.00      1.00      1.00        10
                       ask_bot_name       1.00      1.00      1.00        10
                   ask_contact_info       1.00      1.00      1.00        11
                  ask_discount_info       1.00      1.00      1.00        10
               ask_feedback_process       1.00      1.00      1.00        11
                  ask_location_info       1.00      1.00      1.00        10
                ask_payment_methods       1.00      1.00      1.00         9
               ask_service_duration       1.00      0.90      0.95        10
                  ask_service_price       0.82      0.90      0.86        1

### Inference Intent Classifer Model

In [39]:
def predict_intent(text):
    pred = pipeline.predict([text])[0]
    confidence = pipeline.predict_proba([text]).max()   
    return {"text": text, "intent": pred, "confidence": confidence}

example_inputs = [
    "halo", "saya mau servis mobil", "berapa biaya ganti rem?", "mobil saya mati total",
    "oke, sampai jumpa", "kamu suka bola?", "apa layanan yang tersedia di bengkel?",
    "berapa lama waktu servis rutin?", "kenapa mesin motor panas banget?"
]

for t in example_inputs:
    print(predict_intent(t))

{'text': 'halo', 'intent': 'greeting', 'confidence': 0.47284279163170845}
{'text': 'saya mau servis mobil', 'intent': 'service_booking', 'confidence': 0.6978447188355686}
{'text': 'berapa biaya ganti rem?', 'intent': 'price_inquiry', 'confidence': 0.7026534732663138}
{'text': 'mobil saya mati total', 'intent': 'report_symptom_or_complaint', 'confidence': 0.6683436350368392}
{'text': 'oke, sampai jumpa', 'intent': 'end_or_pause_conversation', 'confidence': 0.9599046785425741}
{'text': 'kamu suka bola?', 'intent': 'off_topic', 'confidence': 0.9588405183739965}
{'text': 'apa layanan yang tersedia di bengkel?', 'intent': 'ask_service_types', 'confidence': 0.369128888641415}
{'text': 'berapa lama waktu servis rutin?', 'intent': 'ask_service_duration', 'confidence': 0.9387221540081647}
{'text': 'kenapa mesin motor panas banget?', 'intent': 'report_symptom_or_complaint', 'confidence': 0.9636777509822799}


### Save Intent Classifier Model

In [40]:
# Jika ingin simpan model dan vectorizer secara terpisah:
vectorizer = pipeline.named_steps['tfidf']
classifier = pipeline.named_steps['svm']

with open("../generated/intent_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("../generated/intent_svm_model.pkl", "wb") as f:
    pickle.dump(classifier, f)

## Dialogue Routing and Routing

### Load Trained Model

In [41]:
intent_model = pickle.load(open("../generated/intent_svm_model.pkl", "rb"))
vectorizer = pickle.load(open("../generated/intent_vectorizer.pkl", "rb"))
encoder = model.get_layer("shared_encoder")
tokenizer = pickle.load(open("../generated/siamese_tokenizer.pkl", "rb"))

### Intent Prediction

In [ ]:
def predict_intent(user_input: str) -> str:
    vector = vectorizer.transform([user_input])
    intent = intent_model.predict(vector)[0]
    return intent

### Intent Retriever

In [ ]:
faq_data = [
    {"question": "Berapa harga jasa service?", "intent": "ask_service_price"},
    {"question": "Saya ingin tahu jadwal bengkel buka", "intent": "ask_workshop_hours"},
    {"question": "Saya ingin memesan layanan", "intent": "start_booking"},
]

def get_embedding(sentence: str) -> np.ndarray:
    return encoder.encode([sentence])[0]

def retrieve_intent(user_input: str, top_k=1):
    input_embedding = get_embedding(user_input)
    similarities = []

    for item in faq_data:
        db_embedding = get_embedding(item["question"])
        sim = np.dot(input_embedding, db_embedding) / (
            np.linalg.norm(input_embedding) * np.linalg.norm(db_embedding)
        )
        similarities.append((item["intent"], sim))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[0][0] if similarities else "fallback"


### Intent Handler

In [ ]:
def handle_intent(intent: str, user_input: str):
    response_map = {
        "greeting": "Hai, ada yang bisa saya bantu?",
        "thank_you": "Sama-sama!",
        "start_booking": "Tentu, mari kita mulai proses booking.",
        "ask_service_price": "Harga layanan kami mulai dari Rp 150.000.",
        "fallback": "Maaf, saya tidak memahami maksud Anda. Bisa ulangi?"
    }

    return response_map.get(intent, "Belum ada handler untuk intent ini.")

### Dialogue Routing

In [ ]:
def route(user_input: str):
    intent = predict_intent(user_input)

    if intent == "fallback":
        intent = retrieve_intent(user_input)

    response = handle_intent(intent, user_input)
    return intent, response

### Simulation

In [ ]:

conversation = [
    "Halo bot!",
    "Saya ingin tahu harga jasa service",
    "Kapan bengkel buka?",
    "Saya ingin booking servis",
    "Terima kasih atas bantuannya",
    "Oh iya, bisa reset jadwal booking saya?",
    "Sampai jumpa!"
]


print("🤖 Bot: Hai! Saya asisten bengkel Anda. Bagaimana saya bisa membantu?\n")

for user_input in conversation:
    intent, response = route(user_input)
    print(f"👤 User: {user_input}")
    print(f"🤖 Bot (intent: {intent}): {response}\n")


🤖 Bot: Hai! Saya asisten bengkel Anda. Bagaimana saya bisa membantu?

👤 User: Halo bot!
🤖 Bot (intent: request_human_agent): Baik, saya akan menghubungkan Anda dengan agen manusia.

👤 User: Saya ingin tahu harga jasa service
🤖 Bot (intent: confirm_booking): Booking Anda telah dikonfirmasi. Terima kasih.

👤 User: Kapan bengkel buka?
🤖 Bot (intent: request_human_agent): Baik, saya akan menghubungkan Anda dengan agen manusia.

👤 User: Saya ingin booking servis
🤖 Bot (intent: report_symptom_or_complaint): Bisa Anda jelaskan lebih detail gejalanya?

👤 User: Terima kasih atas bantuannya
🤖 Bot (intent: cancel_booking): Booking Anda telah dibatalkan sesuai permintaan.

👤 User: Oh iya, bisa reset jadwal booking saya?
🤖 Bot (intent: request_human_agent): Baik, saya akan menghubungkan Anda dengan agen manusia.

👤 User: Sampai jumpa!
🤖 Bot (intent: ask_bot_capabilities): Saya bisa membantu booking, cek harga, dan konsultasi awal kerusakan.



## Dialog Management and Routing

### Load Traned Model

In [ ]:
# Load previously trained models
intent_model = pickle.load(open("../generated/intent_svm_model.pkl", "rb"))
vectorizer = pickle.load(open("../generated/intent_vectorizer.pkl", "rb"))
encoder = model.get_layer("shared_encoder")
tokenizer = pickle.load(open("../generated/siamese_tokenizer.pkl", "rb"))


In [ ]:
MAX_SEQ_LEN      = 30
INTENT_THRESHOLD = 0.6

INTENTS = [
    "greeting", "service_booking", "price_inquiry",
    "complaint", "goodbye", "fallback"
]

ACTIONS = {
    "greeting":          ["respond_greeting"],
    "goodbye":           ["respond_goodbye"],
    "service_booking":   ["retrieve_booking_info"],
    "price_inquiry":     ["retrieve_price_info"],
    "complaint":         ["retrieve_complaint_solution"],
    "fallback":          ["respond_fallback"]
}

CANDIDATES = {
    "retrieve_booking_info": [
        "saya ingin booking servis mobil besok",
        "jadwal servis tersedia hari ini?"
    ],
    "retrieve_price_info": [
        "berapa biaya ganti oli?",
        "berapa harga servis rutin?"
    ],
    "retrieve_complaint_solution": [
        "mobil saya mati total",
        "kenapa mesin cepat panas?"
    ]
}

In [ ]:
def _encode_texts(texts):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=MAX_SEQ_LEN, padding='post')

CANDIDATE_EMB = {
    action: encoder.predict(_encode_texts(sentences))
    for action, sentences in CANDIDATES.items()
}

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

STATIC_RESPONSES = {
    "apology": ["Maaf atas ketidaknyamanannya.", "Mohon maaf, kami akan segera perbaiki."],
    "ask_bot_name": ["Saya AutoServ, asisten servis kendaraan Anda!", "Nama saya AutoServ, senang bertemu!"],
    "ask_bot_capabilities": [
        "Saya bisa bantu booking servis, cek harga, kasih saran troubleshoot, dan banyak lagi. Mau coba apa?",
        "Bisa booking, cek biaya, atau kasih tips perawatan kendaraan. Apa yang Anda butuh?"
    ],
    "ask_contact_info": ["Hubungi kami di 0812-3456-7890 atau email cs@autoserv.id."],
    "ask_discount_info": ["Ada diskon 10% untuk servis pertama, bro! Mau booking sekarang?"],
    "ask_feedback_process": ["Silakan isi feedback di link autoserv.id/feedback atau chat di sini."],
    "ask_location_info": ["Workshop kami di Jl. Merdeka No.10, Jakarta. Buka Senin–Sabtu."],
    "ask_payment_methods": ["Kami terima transfer bank, kartu kredit, dan e-wallet seperti GoPay."],
    "ask_service_duration": ["Servis biasanya 2–3 jam, tergantung jenis layanan."],
    "ask_service_price": ["Harga servis mulai dari Rp200.000, tergantung kendaraan."],
    "ask_service_types": ["Kami punya servis rutin, ganti oli, tune-up, dan perbaikan mesin."],
    "ask_sparepart_price": ["Harga sparepart bervariasi, bisa kasih tahu tipe kendaraan Anda?"],
    "ask_troubleshooting": ["Coba cek level oli dan baterai dulu. Ceritain gejalanya, saya bantu analisis."],
    "ask_weather": ["Hari ini cerah di Jakarta, cocok buat servis! Mau info cuaca kota lain?"],
    "ask_workshop_hours": ["Workshop buka Senin–Sabtu, 08:00–17:00. Minggu tutup."],
    "give_feedback": ["Terima kasih atas feedbacknya! Kami akan terus tingkatkan layanan."],
    "off_topic": ["Haha, seru bro, tapi balik ke bengkel yuk! Mau servis apa?"],
    "price_inquiry": None,  # Ditangani eksternal
    "request_service_advice": ["Ceritain gejala kendaraan Anda, saya bantu kasih saran!"],
    "tell_joke": [
        "Kenapa mobil selalu bahagia? Karena selalu 'gas' terus! 😂",
        "Motor apa yang paling genit? Motor matic, bro!"
    ],
    "thank_you": ["Sama-sama! Senang bisa membantu.", "Anytime, bro!"],
    "request_human_agent": ["Oke, saya hubungkan ke agen manusia. Tunggu sebentar, ya!"],
    "request_repetition_or_clarification": ["Bisa dijelasin lagi, bro? Saya agak bingung."],
}

class DialogueState(Enum):
    INITIAL = "INITIAL"
    ASK_DATE = "ASK_DATE"
    ASK_VEHICLE = "ASK_VEHICLE"
    CONFIRM_BOOKING = "CONFIRM_BOOKING"
    ASK_RESCHEDULE_DATE = "ASK_RESCHEDULE_DATE"
    ASK_CANCEL_ID = "ASK_CANCEL_ID"
    PRICING_DONE = "PRICING_DONE"
    COMPLAINT_DONE = "COMPLAINT_DONE"
    END = "END"
    PAUSED = "PAUSED"

def is_likely_datetime(text):
    if not isinstance(text, str):
        return False
    keywords = ["jam", "besok", "tanggal", "pagi", "siang", "sore",
                "senin", "selasa", "rabu", "kamis", "jumat", "sabtu", "minggu"]
    return any(k in text.lower() for k in keywords)

def is_likely_vehicle(text):
    if not isinstance(text, str):
        return False
    keywords = ["mobil", "motor", "kendaraan", "sedan", "suv", "matic", "bebek"]
    return any(k in text.lower() for k in keywords)

class UserSession:
    def __init__(self):
        self.state = DialogueState.INITIAL
        self.data = {}
        self.last_active = datetime.now()
        self.context = []  # Menyimpan riwayat intent terakhir

    def is_expired(self, timeout_minutes=30):
        return datetime.now() > self.last_active + timedelta(minutes=timeout_minutes)

    def update_activity(self):
        self.last_active = datetime.now()

user_sessions = {}

class DialogueManager:
    def __init__(self, user_id):
        self.user_id = user_id
        self.session = user_sessions.setdefault(user_id, UserSession())
        self.state = self.session.state
        self.data = self.session.data
        self.context = self.session.context

        # Bersihkan sesi jika expired
        if self.session.is_expired():
            logger.info(f"Sesi {user_id} telah expired. Mereset ke INITIAL.")
            self.reset_state()

    def save(self):
        """Simpan state dan data ke sesi."""
        self.session.state = self.state
        self.session.data = self.data
        self.session.context = self.context
        self.session.update_activity()
        user_sessions[self.user_id] = self.session
        logger.debug(f"Sesi {self.user_id} disimpan: state={self.state}, data={self.data}")

    def reset_state(self):
        """Reset sesi ke state awal."""
        self.session = UserSession()
        self.state = self.session.state
        self.data = self.session.data
        self.context = self.session.context
        self.save()
        logger.info(f"Sesi {self.user_id} direset ke INITIAL.")

    def add_context(self, intent):
        """Tambahkan intent ke konteks (maksimal 5 intent terakhir)."""
        self.context.append(intent)
        if len(self.context) > 5:
            self.context.pop(0)

    def suggest_intent(self):
        """Berikan saran intent berdasarkan konteks."""
        if "start_booking" in self.context or "service_booking" in self.context:
            return "Lanjut booking servis, bro? Kapan waktunya?"
        elif "price_inquiry" in self.context:
            return "Mau cek harga servis lagi atau langsung booking?"
        elif "report_symptom_or_complaint" in self.context:
            return "Ada keluhan lain soal kendaraan yang mau diceritain?"
        return "Mau booking servis, cek harga, atau apa nih, bro?"

    def transition(self, intent, user_input=None):
        """Proses transisi state berdasarkan intent dan input pengguna."""
        try:
            # Validasi input
            if user_input is not None and not isinstance(user_input, str):
                logger.warning(f"Input tidak valid untuk user {self.user_id}: {user_input}")
                return "Maaf, input Anda tidak valid. Coba lagi, bro!"

            # Tambahkan intent ke konteks
            self.add_context(intent)
            logger.info(f"User {self.user_id}: Intent={intent}, State={self.state}, Input={user_input}")

            # Priority reset setelah terminal state
            if self.state == DialogueState.END and intent not in {"greeting"}:
                self.reset_state()

            # Booking Flow
            if self.state == DialogueState.INITIAL and intent in {"start_booking", "service_booking"}:
                self.state = DialogueState.ASK_DATE
                self.save()
                return "Kapan Anda ingin melakukan booking servis? (Contoh: besok pagi jam 9)"

            if self.state == DialogueState.ASK_DATE and intent == "provide_booking_date":
                if is_likely_datetime(user_input):
                    self.data["booking_date"] = user_input
                    self.state = DialogueState.ASK_VEHICLE
                    self.save()
                    return "Jenis kendaraan apa yang ingin Anda servis? (Contoh: motor matic, mobil sedan)"
                else:
                    logger.warning(f"Input tanggal tidak valid: {user_input}")
                    return "Maaf, sepertinya tanggalnya tidak jelas. Bisa kasih contoh seperti 'besok jam 10 pagi'?"

            if self.state == DialogueState.ASK_VEHICLE and intent == "provide_vehicle_type":
                if is_likely_vehicle(user_input):
                    self.data["vehicle_type"] = user_input
                    date = self.data.get("booking_date", "tanggal tertentu")
                    vehicle = self.data.get("vehicle_type", "kendaraan")
                    self.state = DialogueState.CONFIRM_BOOKING
                    self.save()
                    return f"Booking {vehicle} pada {date}, benar? (Ketik 'ya' atau 'tidak')"
                else:
                    logger.warning(f"Input kendaraan tidak valid: {user_input}")
                    return "Maaf, jenis kendaraan tidak jelas. Contoh: 'motor matic' atau 'mobil sedan'. Coba lagi!"

            if self.state == DialogueState.CONFIRM_BOOKING and intent == "confirm_yes":
                self.state = DialogueState.END
                self.save()
                return "Booking berhasil! Anda akan dapat konfirmasi via WhatsApp. Terima kasih!"

            if self.state == DialogueState.CONFIRM_BOOKING and intent == "confirm_no":
                self.reset_state()
                return "Baik, booking dibatalkan. Ingin mulai dari awal atau ada pertanyaan lain?"

            # Reschedule Flow
            if self.state == DialogueState.INITIAL and intent == "reschedule_booking":
                self.state = DialogueState.ASK_RESCHEDULE_DATE
                self.save()
                return "Masukkan nomor booking yang ingin diubah jadwalnya (contoh: BK12345)."

            if self.state == DialogueState.ASK_RESCHEDULE_DATE and intent == "provide_booking_date":
                if is_likely_datetime(user_input):
                    self.data["new_date"] = user_input
                    self.state = DialogueState.END
                    self.save()
                    return f"Jadwal booking diubah ke {user_input}. Anda akan dapat konfirmasi baru. Terima kasih!"
                else:
                    logger.warning(f"Input tanggal reschedule tidak valid: {user_input}")
                    return "Tanggalnya kurang jelas, bro. Contoh: 'Senin jam 14:00'. Coba lagi!"

            # Cancel Booking
            if self.state == DialogueState.INITIAL and intent == "cancel_booking":
                self.state = DialogueState.ASK_CANCEL_ID
                self.save()
                return "Masukkan nomor booking yang ingin dibatalkan (contoh: BK12345)."

            if self.state == DialogueState.ASK_CANCEL_ID and intent == "provide_booking_details":
                self.state = DialogueState.END
                self.save()
                return "Booking berhasil dibatalkan. Ada lagi yang bisa saya bantu?"

            # Price Inquiry
            if self.state == DialogueState.INITIAL and intent == "price_inquiry":
                self.state = DialogueState.PRICING_DONE
                self.save()
                return None  # Ditangani oleh retrieve_price_info()

            if self.state == DialogueState.PRICING_DONE and intent == "confirm_yes":
                self.state = DialogueState.ASK_DATE
                self.save()
                return "Keren, kapan mau booking servisnya?"

            if self.state == DialogueState.PRICING_DONE and intent == "confirm_no":
                self.reset_state()
                return "Oke, tidak jadi booking. Mau tanya apa lagi, bro?"

            if self.state == DialogueState.PRICING_DONE:
                return random.choice([
                    "Mau langsung booking layanan ini, bro?",
                    "Lanjut ke booking sekarang atau tanya yang lain?"
                ])

            # Complaint Flow
            if self.state == DialogueState.INITIAL and intent == "report_symptom_or_complaint":
                if user_input and len(user_input.strip()) > 5:  # Validasi panjang input
                    self.data["complaint_text"] = user_input
                    self.state = DialogueState.COMPLAINT_DONE
                    self.save()
                    return None  # Ditangani oleh retrieve_complaint_solution()
                else:
                    logger.warning(f"Input keluhan terlalu pendek: {user_input}")
                    return "Keluhannya kurang jelas, bro. Ceritain lebih detail, misal: 'motor mati tiba-tiba'."

            if self.state == DialogueState.COMPLAINT_DONE and intent == "confirm_no":
                self.reset_state()
                return "Baik, apakah ada hal lain yang bisa saya bantu?"

            if self.state == DialogueState.COMPLAINT_DONE:
                return random.choice([
                    "Perlu bantuan lain soal keluhan ini, bro?",
                    "Ada gejala lain yang mau dilaporkan?"
                ])

            # Pause Conversation
            if intent == "end_or_pause_conversation" and "nanti" in (user_input or "").lower():
                self.state = DialogueState.PAUSED
                self.save()
                return "Oke, saya pause dulu, bro. Bilang kapan mau lanjut lagi!"

            # Resume from Pause
            if self.state == DialogueState.PAUSED and intent not in {"greeting", "end_or_pause_conversation"}:
                self.state = DialogueState.INITIAL
                self.save()
                return "Keren, kita lanjut! Mau ngobrol apa sekarang?"

            # Static Intents
            if intent in STATIC_RESPONSES:
                resp = STATIC_RESPONSES[intent]
                if resp:
                    response = random.choice(resp) if isinstance(resp, list) else resp
                    self.save()
                    return response
                return None  # External handler

            # Greetings & Goodbye
            if intent == "greeting":
                self.reset_state()
                return random.choice([
                    "Halo, bro! Mau servis apa hari ini?",
                    "Hai, selamat datang di AutoServ! Butuh bantuan apa?"
                ])

            if intent == "goodbye" or intent == "end_or_pause_conversation":
                self.reset_state()
                return random.choice([
                    "Makasih, bro! Sampai jumpa lagi!",
                    "Oke, sampai ketemu lagi. Hati-hati di jalan!"
                ])

            # Fallback dengan saran
            if intent == "fallback":
                logger.warning(f"Fallback triggered untuk user {self.user_id}: {user_input}")
                return self.suggest_intent()

            # Default
            logger.error(f"Intent tidak dikenali untuk user {self.user_id}: {intent}")
            return "Maaf, saya bingung, bro. Coba tanya soal servis atau booking, deh!"

        except Exception as e:
            logger.error(f"Error di transition untuk user {self.user_id}: {str(e)}")
            return "Ups, ada masalah teknis. Coba ulang atau hubungi CS kami, bro!"

    @staticmethod
    def load_responses(file_path):
        """Muat respons statis dari file JSON (opsional)."""
        try:
            with open(file_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            logger.error(f"Gagal memuat respons dari {file_path}: {str(e)}")
            return STATIC_RESPONSES

In [ ]:
try:
    from external import encoder, CANDIDATE_EMB, CANDIDATES, route_intent
except ImportError:
    logger.warning("Dependensi eksternal tidak ditemukan. Menggunakan dummy.")
    encoder = lambda x: np.random.rand(1, 128)
    CANDIDATE_EMB = {"retrieve_booking_info": np.random.rand(10, 128),
                     "retrieve_price_info": np.random.rand(10, 128),
                     "retrieve_complaint_solution": np.random.rand(10, 128)}
    CANDIDATES = {"retrieve_booking_info": ["Booking BK12345: 20 Mei 2025, motor matic"],
                  "retrieve_price_info": ["Servis rutin Rp200.000"],
                  "retrieve_complaint_solution": ["Cek baterai untuk motor mati."]}
    def route_intent(text):
        return "fallback", 0.5, ["respond_fallback"]

def _encode_texts(texts):
    """Fungsi dummy untuk encoding teks (sesuaikan dengan encoder sebenarnya)."""
    return texts

def retrieve_generic(action_key, query):
    """Mencari kandidat terbaik untuk action tertentu berdasarkan similarity."""
    try:
        if action_key not in CANDIDATE_EMB or action_key not in CANDIDATES:
            logger.error(f"Action key {action_key} tidak ditemukan.")
            return "Informasi tidak tersedia.", 0.0

        emb_cands = CANDIDATE_EMB[action_key]
        q_emb = encoder.predict(_encode_texts([query]))
        sims = cosine_similarity(q_emb, emb_cands)[0]
        idx = int(np.argmax(sims))
        return CANDIDATES[action_key][idx], float(sims[idx])
    except Exception as e:
        logger.error(f"Error di retrieve_generic untuk {action_key}: {str(e)}")
        return "Maaf, terjadi kesalahan saat mencari informasi.", 0.0

def retrieve_booking_info(query, session):
    """Mengambil informasi booking."""
    try:
        match, sim = retrieve_generic("retrieve_booking_info", query)
        booking_id = session.get("booking_id", "tidak diketahui")
        return f"Info booking (ID: {booking_id}): {match} [Similarity: {sim:.2f}]"
    except Exception as e:
        logger.error(f"Error di retrieve_booking_info: {str(e)}")
        return "Maaf, gagal mengambil info booking. Coba lagi, bro!"

def retrieve_price_info(query, session):
    """Mengambil informasi harga."""
    try:
        match, sim = retrieve_generic("retrieve_price_info", query)
        vehicle_type = session.get("vehicle_type", "kendaraan")
        return f"Harga untuk {vehicle_type}: {match} [Similarity: {sim:.2f}]"
    except Exception as e:
        logger.error(f"Error di retrieve_price_info: {str(e)}")
        return "Maaf, gagal mengambil info harga. Coba tanya lagi, bro!"

def retrieve_complaint_solution(query, session):
    """Mengambil solusi untuk keluhan."""
    try:
        match, sim = retrieve_generic("retrieve_complaint_solution", query)
        complaint = session.get("complaint_text", "keluhan tidak diketahui")
        return f"Solusi untuk '{complaint}': {match} [Similarity: {sim:.2f}]"
    except Exception as e:
        logger.error(f"Error di retrieve_complaint_solution: {str(e)}")
        return "Maaf, gagal menemukan solusi. Ceritain lagi keluhannya, bro!"

def respond_greeting(query, session):
    """Respons untuk sapaan."""
    return random.choice([
        "Halo, bro! Mau servis apa hari ini?",
        "Hai, selamat datang di AutoServ! Butuh bantuan apa?"
    ])

def respond_goodbye(query, session):
    """Respons untuk perpisahan."""
    return random.choice([
        "Makasih, bro! Sampai jumpa lagi!",
        "Oke, hati-hati di jalan! Kapan-kapan ngobrol lagi."
    ])

def respond_pause(query, session):
    """Respons untuk jeda percakapan."""
    return random.choice([
        "Oke, saya pause dulu, bro. Bilang kapan mau lanjut!",
        "Baik, tunggu dulu ya. Kapan lanjut lagi, bro?"
    ])

def respond_fallback(query, session, dm=None):
    """Respons fallback dengan saran kontekstual."""
    try:
        if dm:
            suggestion = dm.suggest_intent()
            return f"Maaf, kurang paham maksudnya. {suggestion}"
        return random.choice([
            "Maaf, saya bingung, bro. Mau tanya soal servis atau booking?",
            "Pertanyaannya agak sulit, nih. Coba jelasin lagi, bro!"
        ])
    except Exception as e:
        logger.error(f"Error di respond_fallback: {str(e)}")
        return "Ups, saya bingung. Coba tanya soal servis, bro!"

# Strategi respons diperbarui
RESPONSE_STRATEGIES = {
    "retrieve_booking_info": retrieve_booking_info,
    "retrieve_price_info": retrieve_price_info,
    "retrieve_complaint_solution": retrieve_complaint_solution,
    "respond_greeting": respond_greeting,
    "respond_goodbye": respond_goodbye,
    "respond_pause": respond_pause,
    "respond_fallback": respond_fallback
}

2025-05-20 17:45:47,189 - WARNING - Dependensi eksternal tidak ditemukan. Menggunakan dummy.


In [ ]:
def run_chatbot(user_id, user_input):
    """Menjalankan logika chatbot."""
    try:
        # Validasi input
        if not user_id or not isinstance(user_id, str):
            logger.error("User ID tidak valid.")
            return {"error": "User ID tidak valid."}
        if not user_input or not isinstance(user_input, str):
            logger.warning(f"Input tidak valid untuk user {user_id}: {user_input}")
            return {
                "intent": "fallback",
                "confidence": 0.0,
                "state": DialogueState.INITIAL.value,
                "response": "Maaf, input Anda kosong atau tidak valid. Coba lagi, bro!"
            }

        logger.info(f"Memproses input untuk user {user_id}: {user_input}")

        # Step 1: Prediksi intent dan confidence
        try:
            intent, confidence, actions = route_intent(user_input)
            logger.debug(f"Intent terdeteksi: {intent}, Confidence: {confidence}, Actions: {actions}")
        except Exception as e:
            logger.error(f"Error di route_intent untuk user {user_id}: {str(e)}")
            intent, confidence, actions = "fallback", 0.0, ["respond_fallback"]

        # Step 2: Inisialisasi DialogueManager
        dm = DialogueManager(user_id)

        # Step 3: Coba FSM terlebih dahulu
        fsm_resp = dm.transition(intent, user_input)

        # Step 4: Jika FSM berhasil menangani
        if fsm_resp is not None:
            dm.save()
            logger.info(f"FSM menangani intent {intent} untuk user {user_id}: {fsm_resp}")
            return {
                "intent": intent,
                "confidence": float(confidence),
                "state": dm.state.value,  # Menggunakan Enum value
                "response": fsm_resp,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S WIB")
            }

        # Step 5: Intent-Driven Strategy Fallback
        action = actions[0] if actions else "respond_fallback"
        handler = RESPONSE_STRATEGIES.get(action, lambda q, s: respond_fallback(q, s, dm))
        try:
            response = handler(user_input, dm.data)
            logger.info(f"Handler {action} menangani untuk user {user_id}: {response}")
        except Exception as e:
            logger.error(f"Error di handler {action} untuk user {user_id}: {str(e)}")
            response = respond_fallback(user_input, dm.data, dm)

        # Step 6: Simpan dan kembalikan hasil
        dm.save()
        return {
            "intent": intent,
            "confidence": float(confidence),
            "state": dm.state.value,
            "action": action,
            "response": response,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S WIB")
        }

    except Exception as e:
        logger.error(f"Error kritis di run_chatbot untuk user {user_id}: {str(e)}")
        return {
            "intent": "fallback",
            "confidence": 0.0,
            "state": DialogueState.INITIAL.value,
            "response": "Ups, ada masalah teknis. Coba lagi atau hubungi CS kami, bro!",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S WIB")
        }

In [ ]:
user_id = "user123"
for q in [
        "halo",
        "berapa biaya ganti rem?",
        "saya mau booking servis motor",
        "besok jam 10 pagi",
        "motor",
        "ya",
        "oke sampai jumpa"
    ]:
        out = run_chatbot(user_id, q)
        print(f"User: {q}")
        print(f"Bot: {out['response']} (state={out['state']})")
        print("—" * 20)

2025-05-20 17:46:17,336 - INFO - Memproses input untuk user user123: halo
2025-05-20 17:46:17,338 - INFO - User user123: Intent=fallback, State=DialogueState.INITIAL, Input=halo
2025-05-20 17:46:17,339 - WARNING - Fallback triggered untuk user user123: halo
2025-05-20 17:46:17,339 - INFO - FSM menangani intent fallback untuk user user123: Mau booking servis, cek harga, atau apa nih, bro?
2025-05-20 17:46:17,340 - INFO - Memproses input untuk user user123: berapa biaya ganti rem?
2025-05-20 17:46:17,340 - INFO - User user123: Intent=fallback, State=DialogueState.INITIAL, Input=berapa biaya ganti rem?
2025-05-20 17:46:17,341 - WARNING - Fallback triggered untuk user user123: berapa biaya ganti rem?
2025-05-20 17:46:17,342 - INFO - FSM menangani intent fallback untuk user user123: Mau booking servis, cek harga, atau apa nih, bro?
2025-05-20 17:46:17,343 - INFO - Memproses input untuk user user123: saya mau booking servis motor
2025-05-20 17:46:17,344 - INFO - User user123: Intent=fallback

User: halo
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: berapa biaya ganti rem?
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: saya mau booking servis motor
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: besok jam 10 pagi
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: motor
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: ya
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————
User: oke sampai jumpa
Bot: Mau booking servis, cek harga, atau apa nih, bro? (state=INITIAL)
————————————————————


In [ ]:
RESPONSE_TEMPLATES = {
    "greeting": {
        "responses": [
            "Halo bro, apa kabar kendaraanmu hari ini?",
            "Selamat pagi! Siap bantu cek mesin atau banmu?",
            "Hai, welcome ke bengkel virtual kami! Apa yang bisa dibantu?",
            "Apa kabar, mekanik andalanmu di sini! Motor atau mobil?",
            "Selamat malam, butuh bantuan darurat buat kendaraanmu?",
            "Hey, apa kabar roda duamu? Siap diservis?",
            "Halo, bengkel online siap 24/7 buatmu!",
            "Assalamualaikum, apa kabar kawan? Kendaraan oke?",
            "Selamat siang, apa kabar rem dan olimu?",
            "Hai, senang ketemu lagi! Mau cek apa hari ini?",
            "Pagi bro, motornya sehat atau minta diservis?",
            "Halo, apa kabar knalpot dan busimu?",
            "Selamat sore, kendaraanmu butuh TLC (Tender Loving Care)?",
            "Hey, apa kabar setir dan suspensimu?",
            "Halo, bengkel kami siap bikin kendaraanmu kinclong lagi!",
            "Selamat datang! Mau tanya servis, harga, atau booking?",
            "Hai, apa kabar karburator atau injeksinya?",
            "Halo bro, kendaraanmu siap ngegas atau butuh perawatan?",
            "Selamat pagi, apa kabar aki dan lampumu?",
            "Hey, apa kabar ban dan velgmu? Siap dicek?",
        ],
        "follow_up": [
            "Mau tanya soal servis, harga, atau cek kendaraan?",
            "Kendaraanmu lagi bermasalah atau cuma butuh cek rutin?",
            "Apa kabar mesinnya? Butuh bantuan apa hari ini?",
            "Motor atau mobil? Ceritain dong apa yang lagi bikin penasaran!"
        ]
    },
    "goodbye": {
        "responses": [
            "Sampai jumpa bro, jaga kendaraanmu ya!",
            "Makasih udah mampir, sampai ketemu di servis berikutnya!",
            "Oke, hati-hati di jalan dan keep the engine running!",
            "Terima kasih, semoga kendaraanmu selalu mulus!",
            "Dadah, balik lagi kalau butuh bantuan bengkel!",
            "Bye, motor atau mobilmu udah siap ngegas lagi!",
            "Sampai nanti, jangan lupa cek oli rutin ya!",
            "Terima kasih banyak, bengkel kami selalu buka buatmu!",
            "Sampai ketemu, semoga banmu ga kempes di jalan!",
            "Makasih bro, kasih bintang lima buat bengkel kami ya!",
            "Selamat malam, semoga kendaraanmu jalan mulus!",
            "Oke, kapan-kapan mampir lagi buat servis rutin!",
            "Baik, kami tunggu kabar baik dari kendaraanmu!",
            "Terima kasih, ajak temenmu servis di sini ya!",
            "Dadah, semoga mesinmu ga ngambek lagi!",
            "Bye, bengkel kami siap bantu kapan aja!",
            "Sampai nanti, jaga knalpot biar ga berisik ya!",
            "Makasih, review positifmu bakal bikin kami semangat!",
            "Sampai ketemu, semoga remmu selalu pakem!",
            "Terima kasih bro, kami jadi andalanmu kan?",
        ],
        "follow_up": [
            "Kalau ada masalah lagi, langsung hubungi kami ya!",
            "Pengen booking servis berikutnya? Kapan nih?",
            "Mau tips perawatan kendaraan? Sini ceritain dulu!",
            "Jangan lupa share pengalamanmu ke temen ya!"
        ]
    },
    "price_inquiry": {
        "responses": [
            "Nih, estimasi biaya buat servis yang kamu tanya:",
            "Berikut rincian harga yang bisa bikin dompetmu tenang:",
            "Ini dia info harga untuk kebutuhan kendaraanmu:",
            "Cekidot, biaya servis yang mungkin cocok buatmu:",
            "Harga untuk servis itu kira-kira begini, bro:",
            "Mau tahu biaya servis? Ini dia perkiraannya:",
            "Nih, daftar harga buat bikin kendaraanmu kinclong:",
            "Berikut estimasi ongkos yang bisa kami bantu:",
            "Ini info biaya yang relevan buat pertanyaanmu:",
            "Harga servisnya gini, masih masuk budget kan?",
            "Cek harga servis di bawah ini, bro, apa pas?",
            "Nih, rincian biaya yang mungkin kamu butuh:",
            "Ini estimasi harga, tinggal pilih yang cocok!",
            "Harga untuk kebutuhanmu kira-kira begini:",
            "Mau tahu ongkosnya? Nih, cek dulu estimasinya:",
        ],
        "retriever_format": [
            "{retrieved} (Similarity: {sim:.2f})",
            "[{label}] {retrieved} [Score: {sim:.2f}]",
            "{retrieved} ({label}, Sim: {sim:.2f})",
            "Hasil pencarian: {retrieved} [Sim: {sim:.2f}]",
            "{label}: {retrieved} (Skor: {sim:.2f})"
        ],
        "follow_up": [
            "Mau langsung booking servis dengan harga ini?",
            "Harganya cocok? Kapan mau ke bengkel?",
            "Butuh penjelasan detail soal biaya ini?",
            "Pengen tahu promo atau diskon buat servis ini?",
            "Mau cek biaya lain, misalnya ganti oli atau ban?"
        ]
    },
    "service_booking": {
        "responses": [
            "Oke bro, ayo kita atur jadwal servisnya!",
            "Booking servis? Kapan waktu yang pas buatmu?",
            "Tentu, kami siap bantu booking slot servis!",
            "Mau servis kendaraan? Hari apa yang kamu mau?",
            "Baik, booking servis bisa diatur. Jam berapa?",
            "Servis motor atau mobil? Kapan kamu free?",
            "Nih, kita bantu booking servis, pilih hari dong!",
            "Oke, slot servis siap diatur. Mau kapan?",
            "Booking servis gampang, kasih tahu hari yang oke!",
            "Mau bikin jadwal servis? Pilih tanggal dulu yuk!",
            "Keren, kita atur servisnya. Hari apa bro?",
            "Servis darurat atau rutin? Kapan waktunya?",
            "Baik, booking servis masuk daftar. Hari apa?",
            "Mau servis cepat? Kasih tahu waktu yang pas!",
            "Oke, kita siapin slot servis. Pilih hari ya!",
        ],
        "retriever_format": [
            "{retrieved} (Similarity: {sim:.2f})",
            "[{label}] {retrieved} [Score: {sim:.2f}]",
            "{retrieved} ({label}, Sim: {sim:.2f})",
            "Hasil pencarian: {retrieved} [Sim: {sim:.2f}]",
            "{label}: {retrieved} (Skor: {sim:.2f})"
        ],
        "follow_up": [
            "Mau di bengkel mana? Ada preferensi lokasi?",
            "Servis apa yang dibutuhin? Cek mesin atau ban?",
            "Punya teknisi favorit atau bebas aja?",
            "Mau slot pagi, siang, atau sore? Kasih tahu ya!",
            "Kendaraan apa yang mau diservis? Motor atau mobil?"
        ]
    },
    "complaint": {
        "responses": [
            "Wah, sorry banget atas kendalanya, bro!",
            "Terima kasih udah lapor, kita cari solusi bareng ya!",
            "Maaf atas ketidaknyamanan, apa yang lagi bikin kesel?",
            "Kami prihatin dengan masalah kendaraanmu, ayo cek!",
            "Oke, ceritain detailnya, kami bantu selesaikan!",
            "Mohon maaf bro, kita identifikasi masalahnya dulu ya!",
            "Kendala itu bikin pusing ya? Nih, kita bantu!",
            "Terima kasih udah kasih tahu, apa yang bikin kendaraanmu rewel?",
            "Sorry banget atas masalahnya, kami siap bantu!",
            "Masalah kendaraan? Tenang, kita urus bareng!",
            "Waduh, ga enak banget ya? Ceritain apa yang salah!",
            "Maaf atas kendalanya, kita cek bareng yuk!",
            "Terima kasih udah lapor, solusinya kita cari!",
            "Kendaraanmu ngambek? Ayo kita bantu perbaiki!",
            "Sorry bro, kita pastikan kendaraanmu balik normal!",
        ],
        "retriever_format": [
            "{retrieved} (Similarity: {sim:.2f})",
            "[{label}] {retrieved} [Score: {sim:.2f}]",
            "{retrieved} ({label}, Sim: {sim:.2f})",
            "Hasil pencarian: {retrieved} [Sim: {sim:.2f}]",
            "{label}: {retrieved} (Skor: {sim:.2f})"
        ],
        "follow_up": [
            "Bisa ceritain gejalanya lebih detail, bro?",
            "Kapan masalah ini mulai? Baru-baru ini?",
            "Kendaraanmu motor atau mobil? Apa yang bikin rewel?",
            "Mau kami atur jadwal cek ke bengkel?",
            "Gejalanya muncul pas apa? Ngegas atau idle?"
        ]
    },
    "fallback": {
        "responses": [
            "Waduh, kayaknya pertanyaanmu bikin aku bingung, bro!",
            "Sorry, aku belum nangkep maksudmu, ulang dong!",
            "Pertanyaan ini agak unik, bisa dijelasin lagi?",
            "Maaf bro, aku ga paham, coba kasih hint!",
            "Hmmm, kayaknya ini di luar radar bengkel, ulang yuk!",
            "Aku bingung nih, maksudnya apa ya? Ceritain lagi!",
            "Sorry, pertanyaanmu ga masuk buku manual, ulang dong!",
            "Wah, ini pertanyaan level dewa, coba jelasin lagi!",
            "Maaf bro, aku kehilangan sinyal, ulang satu kali lagi!",
            "Pertanyaanmu bikin mekanik bingung, coba lagi ya!",
            "Hmmm, ga nyambung ke bengkel, maksudnya apa bro?",
            "Sorry, aku ga ngerti, coba kasih contoh lain!",
            "Waduh, pertanyaanmu bikin mesinku overheat, ulang!",
            "Maaf, ini ga ada di daftar servis, jelasin lagi dong!",
            "Kayaknya aku salah tangkap, coba ulang perlahan!",
        ],
        "follow_up": [
            "Mau tanya soal servis, harga, atau booking bengkel?",
            "Ceritain kendala kendaraanmu, aku bantu kasih solusi!",
            "Pengen tahu apa? Servis, cek mesin, atau lainnya?",
            "Mungkin aku bisa bantu kalau soal motor atau mobil!",
            "Coba tanya soal bengkel, aku pasti jawab!"
        ]
    }
}

In [ ]:
def generate_response(intent, label=None, retrieved=None, sim_score=None):
    template = RESPONSE_TEMPLATES.get(intent, RESPONSE_TEMPLATES["fallback"])
    base_response = random.choice(template["responses"])
    follow_up = random.choice(template.get("follow_up", ["Ada yang bisa saya bantu lagi, bro?"]))
    full_response = base_response

    if sim_score is not None:
        try:
            sim_score_float = float(sim_score)
        except:
            sim_score_float = 0.0

        if sim_score_float >= 0.9:
            if retrieved and "retriever_format" in template:
                format_string = random.choice(template["retriever_format"])
                retrieved_text = format_string.format(
                    label=label or "Info",
                    retrieved=retrieved,
                    sim=sim_score_float
                )
                full_response += f" {retrieved_text}"

        elif 0.7 <= sim_score_float < 0.9:
            if retrieved and "retriever_format" in template:
                disclaimer = " (Saya cukup yakin dengan jawabannya)"
                format_string = random.choice(template["retriever_format"])
                retrieved_text = format_string.format(
                    label=label or "Info",
                    retrieved=retrieved,
                    sim=sim_score_float
                )
                full_response += f"{disclaimer}: {retrieved_text}"

        else:
            return {
                "response": "Pertanyaanmu agak membingungkan nih, bisa dijelasin lebih detail?",
                "follow_up": "Contoh: 'berapa biaya ganti oli motor matic?' atau 'saya mau booking hari Sabtu'"
            }

    return {
        "response": full_response,
        "follow_up": follow_up
    }


In [ ]:
test_inputs = [
    {"intent": "greeting", "sim_score": 0.85},
    {"intent": "price_inquiry", "sim_score": 0.98, "retrieved": "berapa biaya ganti oli?", "label": "Harga Info"},
    {"intent": "complaint", "sim_score": 0.95, "retrieved": "mobil saya mati total", "label": "Solusi"},
    {"intent": "fallback", "sim_score": 0.60},
    {"intent": "service_booking", "sim_score": 1.0, "retrieved": "saya ingin booking servis mobil besok", "label": "Booking Info"},
    {"intent": "goodbye", "sim_score": 0.97}
]

In [ ]:
results = [generate_response(
    intent=inp["intent"],
    label=inp.get("label"),
    retrieved=inp.get("retrieved"),
    sim_score=inp.get("sim_score")
) for inp in test_inputs]

results

[{'response': 'Hai, senang ketemu lagi! Mau cek apa hari ini?',
  'follow_up': 'Apa kabar mesinnya? Butuh bantuan apa hari ini?'},
 {'response': 'Harga untuk kebutuhanmu kira-kira begini: berapa biaya ganti oli? (Similarity: 0.98)',
  'follow_up': 'Pengen tahu promo atau diskon buat servis ini?'},
 {'response': 'Wah, sorry banget atas kendalanya, bro! Hasil pencarian: mobil saya mati total [Sim: 0.95]',
  'follow_up': 'Kapan masalah ini mulai? Baru-baru ini?'},
 {'response': 'Pertanyaanmu agak membingungkan nih, bisa dijelasin lebih detail?',
  'follow_up': "Contoh: 'berapa biaya ganti oli motor matic?' atau 'saya mau booking hari Sabtu'"},
 {'response': 'Baik, booking servis bisa diatur. Jam berapa? saya ingin booking servis mobil besok (Booking Info, Sim: 1.00)',
  'follow_up': 'Kendaraan apa yang mau diservis? Motor atau mobil?'},
 {'response': 'Makasih, review positifmu bakal bikin kami semangat!',
  'follow_up': 'Mau tips perawatan kendaraan? Sini ceritain dulu!'}]

In [ ]:

class DialogueStateManager:
    def __init__(self):
        # Definisi state machine: tiap state punya kemungkinan transisi berikutnya
        self.state_machine = {
            "START": {
                "next": "GREETING"
            },
            "GREETING": {
                "next": {
                    "booking": "BOOKING_DATE",
                    "complaint": "ASK_COMPLAINT",
                    "price_query": "PROVIDE_PRICE"
                }
            },
            "BOOKING_DATE": {
                "next": "BOOKING_VEHICLE_TYPE"
            },
            "BOOKING_VEHICLE_TYPE": {
                "next": "BOOKING_CONFIRM"
            },
            "BOOKING_CONFIRM": {
                "next": "END"
            },
            "ASK_COMPLAINT": {
                "next": "PROVIDE_SOLUTION"
            },
            "PROVIDE_PRICE": {
                "next": "END"
            },
            "PROVIDE_SOLUTION": {
                "next": "END"
            },
            "END": {}
        }
        # Menyimpan sesi tiap user, format: { user_id: { "state": str, "data": dict } }
        self.user_sessions: Dict[str, Dict] = {}

    def get_user_state(self, user_id: str) -> Dict:
        return self.user_sessions.get(user_id, {"state": "START", "data": {}})

    def update_user_state(self, user_id: str, new_state: str, new_data: Optional[Dict] = None):
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = {"state": new_state, "data": new_data or {}}
        else:
            self.user_sessions[user_id]["state"] = new_state
            if new_data:
                self.user_sessions[user_id]["data"].update(new_data)

    def reset_user_state(self, user_id: str):
        if user_id in self.user_sessions:
            self.user_sessions[user_id] = {"state": "START", "data": {}}

    def generate_response(self, user_id: str, user_input: str, detected_intent: Optional[str]) -> str:
        session = self.get_user_state(user_id)
        state = session["state"]
        data = session["data"]

        if state == "START":
            self.update_user_state(user_id, "GREETING")
            return "Halo! Ada yang bisa saya bantu? Mau booking servis, lapor keluhan, atau tanya harga?"

        if state == "GREETING":
            if detected_intent == "booking":
                self.update_user_state(user_id, "BOOKING_DATE")
                return "Kapan kamu ingin melakukan booking servis?"
            elif detected_intent == "complaint":
                self.update_user_state(user_id, "ASK_COMPLAINT")
                return "Ceritakan keluhan motormu."
            elif detected_intent == "price_query":
                self.update_user_state(user_id, "PROVIDE_PRICE")
                return "Untuk servis ini, biayanya sekitar Rp 200.000."
            else:
                return "Maaf, saya belum paham. Bisa ulangi?"

        if state == "BOOKING_DATE":
            # Simpan tanggal booking
            self.update_user_state(user_id, "BOOKING_VEHICLE_TYPE", {"booking_date": user_input})
            return "Jenis kendaraan apa yang ingin kamu servis?"

        if state == "BOOKING_VEHICLE_TYPE":
            self.update_user_state(user_id, "BOOKING_CONFIRM", {"vehicle_type": user_input})
            date = data.get("booking_date", "<tanggal tidak diketahui>")
            return f"Booking kamu untuk {user_input} pada tanggal {date}, benar kan?"

        if state == "BOOKING_CONFIRM":
            if user_input.strip().lower() == "iya":
                self.update_user_state(user_id, "END")
                return "Booking berhasil! Terima kasih sudah menggunakan layanan kami."
            else:
                self.update_user_state(user_id, "BOOKING_DATE")
                return "Oke, silakan ulangi tanggal booking."

        if state == "ASK_COMPLAINT":
            self.update_user_state(user_id, "PROVIDE_SOLUTION", {"complaint": user_input})
            return "Terima kasih sudah informasikan keluhan. Mekanik kami akan segera cek dan hubungi kamu."

        if state == "PROVIDE_PRICE":
            self.update_user_state(user_id, "END")
            return "Jika ada pertanyaan lain, silakan tanyakan saja ya!"

        if state == "PROVIDE_SOLUTION":
            self.update_user_state(user_id, "END")
            return "Apakah ada hal lain yang bisa saya bantu?"

        if state == "END":
            self.reset_user_state(user_id)
            return "Terima kasih sudah berinteraksi dengan kami. Sampai jumpa!"

        # Default fallback response
        return "Maaf, terjadi kesalahan. Bisa coba ulangi?"

In [ ]:
dialog_manager = DialogueStateManager()

user_id = "user123"

print(dialog_manager.generate_response(user_id, "", None))
# Halo! Ada yang bisa saya bantu? Mau booking servis, lapor keluhan, atau tanya harga?

print(dialog_manager.generate_response(user_id, "Saya mau booking", "booking"))
# Kapan kamu ingin melakukan booking servis?

print(dialog_manager.generate_response(user_id, "Besok jam 10 pagi", None))
# Jenis kendaraan apa yang ingin kamu servis?

print(dialog_manager.generate_response(user_id, "Motor", None))
# Booking kamu untuk Motor pada tanggal Besok jam 10 pagi, benar kan? (ya/tidak)

print(dialog_manager.generate_response(user_id, "iya", None))
# Booking berhasil! Terima kasih sudah menggunakan layanan kami.

print(dialog_manager.generate_response(user_id, "Terima kasih", None))
# Terima kasih sudah berinteraksi dengan kami. Sampai jumpa!


Halo! Ada yang bisa saya bantu? Mau booking servis, lapor keluhan, atau tanya harga?
Kapan kamu ingin melakukan booking servis?
Jenis kendaraan apa yang ingin kamu servis?
Booking kamu untuk Motor pada tanggal Besok jam 10 pagi, benar kan?
Booking berhasil! Terima kasih sudah menggunakan layanan kami.
Terima kasih sudah berinteraksi dengan kami. Sampai jumpa!
